In [ ]:
import numpy as np
from time import time
from KKMeans import KKMeans

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_circles
from sklearn.cluster import KMeans

In [ ]:
n_c = 3
n_s = 10000
n_features = 1
kkm = KKMeans(n_clusters=n_c, init = "random", n_init=1, max_iter=100, algorithm="lloyd", verbose=False, rng=1, q_metric="inertia")
data, _= make_blobs(n_s, n_features, centers=n_c, random_state=0)
start = time()
kkm.fit(data)
end = time()
print(end - start)

In [ ]:
n_samples = 500
n_features = 3
n_clusters = 300
iters = 20
for seed in range(iters):
    data, labels = make_blobs(n_samples, n_features, centers=n_clusters, random_state=seed)
    kkm = KKMeans(n_clusters=n_clusters, init="random", algorithm="lloyd", rng=seed)
    kkme = KKMeans(n_clusters=n_clusters, init="random", algorithm="elkan", rng=seed)
    kkm.fit(data)
    kkme.fit(data)
    print(seed)
    assert all(kkm.labels_ == kkme.labels_)
    assert np.isclose(kkm.quality_, kkme.quality_)
    assert np.isclose(kkm.quality_, kkme.quality_)



In [ ]:
kkm = KKMeans(n_clusters=100, init = "kmeans++", n_init=10, max_iter=100, algorithm="lloyd", verbose=False, rng=0, q_metric="inertia")

In [ ]:
kkm.fit(data)

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=n_clusters, init="random")
km.fit(data)

In [ ]:
km.inertia_

In [ ]:
kkm = KKMeans(n_clusters = 5, verbose = False, init = "kmeans++", kernel = "linear", rng = 0, q_metric="silhouette", n_init=3, tol = 1e-4, algorithm="lloyd")

In [ ]:
x, l = make_circles(7000, factor = 0.4, random_state=0)

In [ ]:
kkm = KKMeans(n_clusters = 2, verbose = True, init = "kmeans++", kernel = "rbf", rng = 0, tol = 1e-4, variance=0.4, n_init = 5, q_metric="inertia", algorithm="lloyd")

In [ ]:
start = time()
kkm.fit(x)
end = time()
print(end - start)
visualize_kkm((x, kkm.labels_)

In [ ]:
visualize_kkm((x, kkm.predict(x))

In [ ]:
all(kkm.predict(x) == kkm.labels_)

In [ ]:
n_c = 75
size = 10000
r_s = 0
x, labels, centers = make_blobs(size, centers=n_c, return_centers = True, random_state = r_s, n_features = 2)

In [ ]:
seed = 0
tol = 1e-4
inits= 3
initmethod = "kmeans++"
kkml =  KKMeans(n_clusters = n_c, verbose = True, init =initmethod, kernel = "linear", rng=seed, tol=tol, n_init=inits, q_metric="inertia", algorithm="lloyd")
kkme =  KKMeans(n_clusters = n_c, verbose = True, init =initmethod, kernel = "linear", rng=seed, tol=tol, n_init=inits, q_metric="inertia", algorithm="elkan")

In [ ]:
start = time()
kkml.fit(x)
end = time()
print(end - start)
visualize_kkm((x, kkml.labels_)

In [ ]:
start = time()
kkme.fit(x)
end = time()
print(end - start)
visualize_kkm((x, kkme.labels_)

In [ ]:
print(all(kkme.labels_ == kkml.labels_))

In [ ]:
iter = 20
elkan_store = []
lloyd_store = []

n_c = 20
size = 2000

tol = 1e-4
inits= 3
initmethod = "kmeans++"
verb = False
max_iter = 100

for i in range(iter):
    print(i)
    x, labels, centers = make_blobs(size, centers=n_c, return_centers = True, random_state = i, n_features = 2)

    kkml =  KKMeans(n_clusters = n_c, verbose=verb, init =initmethod, kernel = "linear", rng=i, tol=tol, n_init=inits, q_metric="inertia", algorithm="lloyd", max_iter=max_iter)
    kkme =  KKMeans(n_clusters = n_c, verbose=verb, init =initmethod, kernel = "linear", rng=i, tol=tol, n_init=inits, q_metric="inertia", algorithm="elkan", max_iter = max_iter)

    start = time()
    kkml.fit(x)
    end = time()
    lloyd_store.append(end - start)
    start = time()
    kkme.fit(x)
    end = time()
    elkan_store.append(end - start)
    assert(all(kkml.labels_ == kkme.labels_))
    assert(np.isclose(kkml.quality_, kkme.quality_))
    

print(sum(elkan_store)/len(elkan_store))
print(sum(lloyd_store)/len(lloyd_store))

In [ ]:
print(sum(elkan_store)/len(elkan_store))
print(sum(lloyd_store)/len(lloyd_store))

In [ ]:
visualize_kkm((x, kkml.labels_)

In [ ]:
visualize_kkm((x, labels)

In [ ]:
help(KKMeans)